# Simple Algorithm to play a game of Tic-Tac-Toe

In [163]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

C:\Users\KasparLazar\AppData\Local\Temp\ipykernel_11724\495279268.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


The code is downloaded from here: https://github.com/doctorsmonsters/minimax_tic_tac_toe/blob/main/Minimax%20TicTacToe.ipynb
I do take any responsibility for its inner working. 

Your task is to understand it, evaluate it, and make it working. 

There can be problems in the code. Try fixing them, as part of setting-up the test cases activity
        

## Board Definition

In [164]:
"""
Notes:
"""

"\nQuestions:\n1. How should I handle the arguments for the _move method in the game. For safety I would expect that we want a Game object to action the move over a User object.\nAs a result the Game object would need to pass the User object the necessary arguments to User object's move method. How should we handle this?\n\n2. Is it acceptable to implement abstract methods as defaulting to redundancy rather than requiring explicit implementation?\nFor instance my implementation of the _check_draw method in the Game class defaults to redundancy by always returning False if not explicitly overriden.\n^ Would this be preferred over pure-abstract as, for instances, if I wanted to apply an algorithm tailored for one game, that has draws as a possible end-game\ncondition, to another game, that does not have a possibility of draws, then the algorithm will not require modification to apply to the second game as \nit has redundant implementation of check_draw over none.\n"

In [2]:
import random
from abc import ABC, abstractmethod

In [166]:
class Game(ABC):
    @abstractmethod
    def __init__(self, players, width, height, board = None):
        pass
    
    @property
    def width(self): return self._width

    @property
    def height(self): return self._height

    @property
    def board(self): return self._board

    @abstractmethod
    def _check_draw(self) -> bool:
        """Return True if the game has reached a draw"""
        pass
    
    @abstractmethod
    def _check_win(self) -> tuple[bool]:
        """Return True and the Winner's symbol if the game has been won"""
        pass
    
    @abstractmethod
    def _space_is_free(self, position) -> bool:
        """Check if the position is open"""
        pass
        
    @abstractmethod
    def _move(self, player, position) -> bool:
        """Action player's move
        Return if the action was successful"""
        pass
    
    @abstractmethod
    def print_board(self) -> None:
        """Visualise the board"""
        pass
    
    @abstractmethod
    def play(self) -> None:
        """Game loop"""
        pass

In [167]:
class Tic_Tac_Toe(Game):
    def __init__(self, players, width = 3, height = 3, row = 3, board = None):
        self._board = {i : ' ' for i in range(1, width*height + 1)} if board == None else board
        self._width = width
        self._height = height
        self._players = players
        self._row = row # the number of the same thing you have to get in a row to win
    
    @property
    def row(self): return self._row
        
    def _check_draw(self) -> bool:
        for square in range(1,self._width*self._height + 1):
            if self._board[square] == ' ':
                return False
        return True
    
    def _check_win(self) -> tuple[bool, str]:
        # check rows for win
        if self._row <= self._width:
            for current_row in range(1, self._width * self._height, self._width):
                for square in range(current_row, current_row + self._width - (self._row - 1)):
                    state = self._board[square]
                    if state == ' ': continue

                    # create a set of the states of the adjacent squares in the row
                    row_states = {self._board[adjacent] for adjacent in range(square, square + self._row)}
                    if row_states == {state}:
                        return True, state
                    
        # check columns for win
        if self._row <= self._height:
            for current_column in range(1, self._width + 1):
                for square in range(current_column, self._width * self._height - (self._width * (self._row - 1)) + current_column, self._width):
                    state = self._board[square]
                    if state == ' ': continue

                    # create a set of states of the adjacent squares in the column
                    column_states = {self._board[adjacent] for adjacent in range(square, square + self._width * self._row, self._width)}
                    if column_states == {state}:
                        return True, state

        # check diagnols for win
        if self._row <= self._width and self._row <= self._height:
            # check down diagnols for win
            for current_diagnol in range(1, (self._width + 1) - (self._row - 1)):
                for square in range(current_diagnol, self._width * self._height - (self._width * (self._row - 1)) + current_diagnol, self._width):
                    state = self._board[square]
                    if state == ' ': continue

                    # create a set of states of the adjacent squares in the column
                    diagnol_states = {self._board[adjacent] for adjacent in range(square, (square + self._row) + self._width * self._row, self._width + 1)}
                    if diagnol_states == {state}:
                        return True, state

            # check up diagnols for win
            for current_diagnol in range(self._width, self._row - 1, -1):
                for square in range(current_diagnol, self._width * self._height - (self._width * (self._row - 1)) + current_diagnol, self._width):
                    state = self._board[square]
                    if state == ' ': continue

                    # create a set of states of the adjacent squares in the column
                    diagnol_states = {self._board[adjacent] for adjacent in range(square, (square - self._row) + (self._width * self._row), self._width - 1)}
                    if diagnol_states == {state}:
                        return True, state

        return False, None
    
    def _space_is_free(self, position) -> bool:
        """Check if the position is open"""
        if (self._board[position]==' '):
            return True
        else:
            return False
        
    def _move(self, player, position) -> bool:
        """Action player's move
        Return if the action was successful"""
        if (self._space_is_free(position)):
            self._board[position] = player
            return True
        else:
            return False
    
    def print_board(self) -> None:
        """Visualise the board"""
        print_str = ''
        for row in range(self._height):
            for square in range(row * self._width + 1, (row + 1) * self._width + 1):
                print_str += str(self._board[square]) + '|'
            print_str = print_str[:-1] + '\n'
            print_str += '-+'*self._width # add the line between rows
            print_str = print_str[:-1] + '\n'
        print_str = print_str[:-(len('-+')*self._width)] # remove excess row lines
        print(print_str)
        return None
    
    def play(self) -> None:
        """Game loop"""
        self.print_board()
        run = True
        while run:
            for player in self._players:
                # get the player's move
                success = self._move(player, player.move(self.board))
                while not success:
                    success = self._move(player, player.move(self.board))
                
                self.print_board()
                
                # check if game ended
                status, winner = self._check_win()
                if status:
                    print('{} Has Won!'.format(winner))
                    run = False
                    break
                elif self._check_draw():
                    print('Draw!')
                    run = False
                    break

In [168]:
class Algorithm(ABC):

    @abstractmethod
    def run(self, board, me):
        """Run the algorithm and return the result"""
        pass

In [169]:
class my_algorithm(Algorithm):
    
    def run(self, board, me) -> int:
        weights = {square : 0 for square in board.keys()}
        bot_squares = {square for square, state in board.items() if state == me}
        
        if len(bot_squares) == 0: return random.choice([square for square in board.keys() if board[square] == ' '])
        
        for square in bot_squares:
            # increase weight of empty squares on current square's row
            for sqr in range((square - 1)//3 * 3 + 1, (square - 1)//3 * 3 + 3 + 1):
                if board[sqr] == ' ':
                    weights[sqr] += 1

            # increase weight of empty squares on current square's column
            for sqr in range((square - 1) % 3 + 1, (square - 1) % 3 + 3 * 2 + 1 + 1):
                if board[sqr] == ' ':
                    weights[sqr] += 1

            # increase weight of squares on the same diagonal as the current square
            diag_squares = {1,5,9} if square in {1,5,9} else set()
            diag_squares |= {3,5,7} if square in {3,5,7} else set()
            for sqr in diag_squares:
                if board[sqr] == ' ':
                    weights[sqr] += 1
        
        # get the best square
        best_square = 0
        best_weight = 0
        for square, weight in weights.items():
            if weight > best_weight:
                best_square, best_weight = square, weight
        return best_square

In [170]:
class minimax(Algorithm):
    
    def run(self, board, me):
        return random.choice([square for square in board.keys() if board[square] == ' '])

In [171]:
class reinforced_learning(Algorithm):
    
    def run(self, board, me):
        return random.choice([square for square in board.keys() if board[square] == ' '])

In [172]:
class Player(ABC):
    @abstractmethod
    def __init__(self, symbol: str):
        self._symbol = symbol
    
    @property
    def symbol(self): return self._symbol
    
    @abstractmethod
    def move(self):
        """Return player's move"""
        pass

    @abstractmethod
    def __str__(self):
        return self._symbol
    
    @abstractmethod
    def __eq__(self, __value: object) -> bool:
        return str(self) == str(__value)
    
    @abstractmethod
    def __ne__(self, __value: object) -> bool:
        return not self.__eq__(__value)

In [173]:
class Computer(Player):
    def __init__(self, symbol: str, algorithm: Algorithm):
        super().__init__(symbol)
        self._algorithm = algorithm

    def move(self, board):
        move = self._algorithm.run(board, self)
        return move

class User(Player):
    def __init__(self, symbol: str):
        super().__init__(symbol)
    
    def move(self, board):
        move = int(input('Enter position for {}: '.format(self._symbol)))
        return move

In [174]:
class Piece(ABC):
    @abstractmethod
    def __init__(self, player, position) -> None:
        pass
    
    @property
    def player(self): return self._player

    @property
    def player(self): return self._position

    @abstractmethod
    def move(self, position, board) -> bool:
        """Return if piece can move to position"""
        pass

    @abstractmethod
    def __str__(self) -> str:
        pass

In [ ]:
class AbstractPieceFactory(ABC):
    
    @abstractmethod
    def factoryMethod(self, player, position, direction = None):
        pass

In [175]:
class Knight(Piece):
    def __init__(self, player, position) -> None:
        self._player = player
        self._position = position
    
    def move(self, position, board) -> bool:
        self._position = position
        return True
    
    def __str__(self):
        return str(self._player) + 'H'

In [176]:
class Rook(Piece):
    def __init__(self, player, position) -> None:
        self._player = player
        self._position = position
    
    def move(self, position, board) -> bool:
        self._position = position
        return True
    
    def __str__(self):
        return str(self._player) + 'R'

In [177]:
class Queen(Piece):
    def __init__(self, player, position) -> None:
        self._player = player
        self._position = position
    
    def move(self, position, board) -> bool:
        self._position = position
        return True
    
    def __str__(self):
        return str(self._player) + 'Q'

In [178]:
class King(Piece):
    def __init__(self, player, position) -> None:
        self._player = player
        self._position = position
    
    def move(self, position, board) -> bool:
        self._position = position
        return True
    
    def __str__(self):
        return str(self._player) + 'K'

In [179]:
class Pawn(Piece):
    def __init__(self, player, position, direction: int) -> None:
        self._player = player
        self._position = position
        self._direction = direction
    
    def move(self, position, board) -> bool:
        self._position = position
        return True
    
    def __str__(self):
        return str(self._player) + 'P'

In [180]:
class Bishop(Piece):
    def __init__(self, player, position) -> None:
        self._player = player
        self._position = position
    
    def move(self, position, board) -> bool:
        self._position = position
        return True
    
    def __str__(self):
        return str(self._player) + 'B'

In [ ]:
class DefaultChessBoardFactory(AbstractPieceFactory):

    def factoryMethod(self, player1: Player, player2: Player) -> dict[int : Piece]:
        board = [Rook(player1,1),Knight(player1,2),Bishop(player1,3),Queen(player1,4),King(player1,5),Bishop(player1,6),Knight(player1,7),Rook(player1,8),
         Pawn(player1,9,-1),Pawn(player1,10,-1),Pawn(player1,11,-1),Pawn(player1,12,-1),Pawn(player1,13,-1),Pawn(player1,14,-1),Pawn(player1,15,-1),Pawn(player1,16,-1),
         None,None,None,None,None,None,None,None,
         None,None,None,None,None,None,None,None,
         None,None,None,None,None,None,None,None,
         None,None,None,None,None,None,None,None,
         Pawn(player2,49,-1),Pawn(player2,50,-1),Pawn(player2,51,1),Pawn(player2,52,1),Pawn(player2,53,1),Pawn(player2,54,1),Pawn(player2,55,1),Pawn(player2,56,1),
         Rook(player2,57),Knight(player2,58),Bishop(player2,59),Queen(player2,60),King(player2,61),Bishop(player2,62),Knight(player2,63),Rook(player2,64)]
        board = {i + 1 : board[i] for i in range(len(board))}
        return board

In [186]:
class Chess(Game):
    def __init__(self, players, width = 8, height = 8, board = None):
        self._board = DefaultChessBoardFactory.factoryMethod(players[0],players[1]) if board == None else board
        self._width = width
        self._height = height
        self._players = players
    
    def print_board(self) -> None:
        print_str = ''
        for row in range(self._height):
            for square in range(row * self._width + 1, (row + 1) * self._width + 1):
                print_str += str(self._board[square]) + '|' if self._board[square] != None else '  ' + '|'
            print_str = print_str[:-1] + '\n'
            print_str += '--+'*self._width # add the line between rows
            print_str = print_str[:-1] + '\n'
        print_str = print_str[:-(len('--+')*self._width)] # remove excess row lines
        print(print_str)
        return None
    
    def _check_draw(self) -> bool:
        """Return True if the game has reached a draw"""
        players_pieces = [0 for _ in self._players]
        for i in range(len(self._players)):
            player = self._players[i]
            for square in self._board.values():
                if square == None: continue
                if square.player == player:
                    players_pieces[i] += 1
        player_pieces = list(set(player_pieces))
        if len(player_pieces) == 1 and player_pieces[0] == 1:
            return True
        return False
    
    def _check_win(self) -> tuple[bool, Player]:
        """Return True and the Winner's symbol if the game has been won"""
        for player in self._players:
            has_king = False
            for piece in self._board.values():
                if piece == None: continue
                if piece.player == player and type(piece) == King:
                    has_king == True
                    break
            if not has_king:
                return True, player
        return False, None

    def _space_is_free(self, position) -> bool:
        """Check if the position is open"""
        if self._board[position] == None:
            return True
        return False
        
    def _move(self, player, position_from, position_to) -> bool:
        """Action player's move
        Return if the action was successful"""
        piece = self._board[position_from] 
        if piece == None: return False
        if piece.player != player: return False
        if piece.move(position_to, self.board) == False: return False
        self._board[position_to] = piece
        self._board[position_from] = None
        return True

    def play(self) -> None:
        """Game loop"""
        self.print_board()
        run = True
        while run:
            for player in self._players:
                # get the player's move
                move_from, move_to = player.move(self.board)
                success = self._move(player, move_from, move_to)
                while not success:
                    move_from, move_to = player.move(self.board)
                    success = self._move(player, move_from, move_to)
                    
                self.print_board()
                
                # check if game ended
                status, winner = self._check_win()
                if status:
                    print('{} Has Won!'.format(winner))
                    run = False
                    break
                elif self._check_draw():
                    print('Draw!')
                    run = False
                    break

In [182]:
class Chess_User(Player):
    def __init__(self, symbol: str) -> None:
        super().__init__(symbol)
    
    def move(self, board):
        move_from = int(input('Enter position of piece to move for Player {}: '.format(self._symbol)))
        move_to = int(input('Enter position to move {} piece to: '.format(board[move_from])))
        return move_from, move_to 

In [183]:
class Chess_Computer(Player):
    def __init__(self, symbol: str, algorithm: Algorithm):
        super().__init__(symbol)
        self._algorithm = algorithm

    def move(self, board):
        move_from, move_to = self._algorithm.run(board, self)
        return move_from, move_to

In [184]:
class random_algorithm(Algorithm):
    
    def run(self, board, me):
        my_pieces = []
        other_pieces = []
        for key, piece in board.items():
            if piece == None: 
                other_pieces.append(key)
                continue
            if piece.player != me:
                other_pieces.append(key)
            else:
                my_pieces.append(key)
        move_from = random.choice(my_pieces)
        move_to = random.choice(other_pieces)
        return move_from, move_to

In [ ]:
class Backgammon(Game):
    def __init__(self, players, width = 24, height = 2, board=None):
        board = board if board != None else {i + 1 : None for i in range(width)} | {
            1 : (players[0], 2), 12 : (players[0], 5), 17 : (players[0], 3), 19 : (players[0], 5),
            24 : (players[1], 2), 13 : (players[1], 5), 8 : (players[0], 3), 6 : (players[1], 5)
        }
        super().__init__(players, width, height, board)
        self._dice = lambda : tuple(random.randrange(1,6), random.randrange(1,6))
    
    def _check_draw(self) -> bool:
        """Return True if the game has reached a draw"""
        False
    
    def _check_win(self) -> tuple[bool]:
        """Return True and the Winner's symbol if the game has been won"""
        pass

    def _space_is_free(self, position) -> bool:
        """Check if the position is open"""
        pass
        
    def _move(self, player, position) -> bool:
        """Action player's move
        Return if the action was successful"""
        pass
    
    def print_board(self) -> None:
        """Visualise the board"""
        pass

    def play(self) -> None:
        """Game loop"""
        pass

In [185]:
def main():
    """opponents = [User('X'), Computer('O',my_algorithm()), Computer('Z',minimax())]
    TicTacToe = Tic_Tac_Toe(opponents)
    TicTacToe.play()"""

    opponents = [Chess_User('1'), Chess_Computer('2',random_algorithm())]
    chess_game = Chess(opponents)
    chess_game.play()

if __name__ == '__main__':
    main()

1R|1H|1B|1Q|1K|1B|1H|1R
--+--+--+--+--+--+--+--
1P|1P|1P|1P|1P|1P|1P|1P
--+--+--+--+--+--+--+--
  |  |  |  |  |  |  |  
--+--+--+--+--+--+--+--
  |  |  |  |  |  |  |  
--+--+--+--+--+--+--+--
  |  |  |  |  |  |  |  
--+--+--+--+--+--+--+--
  |  |  |  |  |  |  |  
--+--+--+--+--+--+--+--
2P|2P|2P|2P|2P|2P|2P|2P
--+--+--+--+--+--+--+--
2R|2H|2B|2Q|2K|2B|2H|2R

  |1H|1B|1Q|1K|1B|1H|1R
--+--+--+--+--+--+--+--
1P|1P|1P|1P|1P|1P|1P|1P
--+--+--+--+--+--+--+--
1R|  |  |  |  |  |  |  
--+--+--+--+--+--+--+--
  |  |  |  |  |  |  |  
--+--+--+--+--+--+--+--
  |  |  |  |  |  |  |  
--+--+--+--+--+--+--+--
  |  |  |  |  |  |  |  
--+--+--+--+--+--+--+--
2P|2P|2P|2P|2P|2P|2P|2P
--+--+--+--+--+--+--+--
2R|2H|2B|2Q|2K|2B|2H|2R

1 Has Won!
